In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

def plot_ansi67(fault_phase, amt, pickup_current, direction, current_magnitude, current_angle, save_image):
    # Definição das tensões trifásicas de fase (sequência direta ABC)
    phase_voltages = {'A': np.exp(1j * 0), 'B': np.exp(1j * -2*np.pi/3), 'C': np.exp(1j * 2*np.pi/3)}
    
    # Definição correta das tensões de polarização
    polarization_mapping = {'A': ('Vbc', phase_voltages['B'] - phase_voltages['C']),
                            'B': ('Vca', phase_voltages['C'] - phase_voltages['A']),
                            'C': ('Vab', phase_voltages['A'] - phase_voltages['B'])}
    polarization_voltage, pol_voltage = polarization_mapping[fault_phase]
    
    # Rotacionando os fasores para que a tensão de polarização fique em 0 graus
    rotation_factor = np.conj(pol_voltage) / abs(pol_voltage)  # Rotação para alinhar a tensão de polarização em 0°
    rotated_voltages = {phase: voltage * rotation_factor for phase, voltage in phase_voltages.items()}
    
    # Criando círculo unitário para visualização da zona de atuação
    theta = np.linspace(0, 2 * np.pi, 300)
  # Criando círculo em função da corrente de pick-up
    x_circle = pickup_current * np.cos(theta)
    y_circle = pickup_current * np.sin(theta)

    
    # Ângulo de máxima sensibilidade (AMT) contado a partir da tensão de polarização
    amt_rad = np.radians(amt)
    
    # Definição da região de atuação (forward ou backward)
    if direction == 'Forward':
        theta_trip = np.linspace(amt_rad - np.pi/2, amt_rad + np.pi/2, 100)
    else:  # Backward
        theta_trip = np.linspace(amt_rad + np.pi/2, amt_rad + 3*np.pi/2, 100)
    
    x_trip = pickup_current * np.cos(theta_trip)
    y_trip = pickup_current * np.sin(theta_trip)
    
    # Ajustando o ângulo da corrente inserida para ser relativo à tensão de fase
    # Calculamos o ângulo da corrente em relação à fase de falta (fault_phase)
    fault_phase_angle = np.angle(rotated_voltages[fault_phase])  # ângulo da tensão de fase
    current_angle_rad = np.radians(current_angle)  # ângulo da corrente inserida
    relative_current_angle = fault_phase_angle + current_angle_rad  # Ajustando a corrente com base no ângulo da fase de falta
    
    # Definição da corrente inserida com base no ângulo relativo
    current_vector = current_magnitude * np.exp(1j * relative_current_angle)
    rotated_current = current_vector  # Corrente já está rotacionada, sem mais rotação
    
    # Cálculo do vetor ortogonal ao AMT
    ortho_angle = amt_rad + np.pi / 2  # Perpendicular ao vetor AMT
    x_ortho = np.array([-1.5, 1.5])  # Limites da reta ortogonal
    y_ortho = np.array([np.tan(ortho_angle) * x for x in x_ortho])  # Equação da reta ortogonal
    
    # Plot
    fig, ax = plt.subplots(figsize=(10,10))
    ax.plot(x_circle, y_circle, 'k--', alpha=0.5, label='Círculo Unitário')
    ax.fill(x_trip, y_trip, 'r', alpha=0.3, label='Limiar de Atuação (Ip)')
    ax.arrow(0, 0, pickup_current * np.cos(amt_rad), pickup_current * np.sin(amt_rad), 
             head_width=0.1, head_length=0.1, fc='b', ec='b', label='AMT')
    
    # Plotando as tensões de fase rotacionadas
    for phase, voltage in rotated_voltages.items():
        ax.arrow(0, 0, voltage.real, voltage.imag, head_width=0.05, head_length=0.05, fc='gray', ec='gray', alpha=0.7)
        ax.text(1.1 * voltage.real, 1.1 * voltage.imag, f'V{phase}', fontsize=12, color='gray')
    
    # Destacando a tensão de polarização (agora fixa em 0°)
    ax.arrow(0, 0, 1, 0, head_width=0.05, head_length=0.05, fc='purple', ec='purple', label=f'Tensão de Polarização ({polarization_voltage})')
    ax.text(1.1, 0.1, polarization_voltage, fontsize=12, color='purple')
    
    # Inserindo o vetor de corrente
    ax.arrow(0, 0, rotated_current.real, rotated_current.imag, head_width=0.05, head_length=0.05, fc='green', ec='green', label='Corrente Inserida')
    ax.text(1.1 * rotated_current.real, 1.1 * rotated_current.imag, 'I', fontsize=12, color='green')
    
    # Plotando a reta ortogonal ao vetor AMT
    ax.plot(x_ortho, y_ortho, 'm--', label='Reta Ortogonal ao AMT')  # Linha ortogonal
    
    # Configuração do gráfico
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_xlabel('Componente Real')
    ax.set_ylabel('Componente Imaginária')
    ax.set_title(f'Proteção ANSI 67 - Falta na Fase {fault_phase} ({polarization_voltage})')
    ax.legend()
    ax.grid()
    
    # Salvando a imagem em alta qualidade se solicitado
    if save_image:
        fig.savefig('proteção_ansi67.png', dpi=300, bbox_inches='tight')
        print("Imagem salva com sucesso como 'proteção_ansi67.png'.")
    
    plt.show()

# Widgets interativos
interact(plot_ansi67, 
         fault_phase=widgets.Dropdown(options=['A', 'B', 'C'], value='A', description='Fase de Falta'),
         amt=widgets.IntSlider(min=-90, max=90, step=5, value=0, description='AMT (°)'),
         pickup_current=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=0.5, description='Pick-up (pu)'),
         direction=widgets.RadioButtons(options=['Forward', 'Backward'], value='Forward', description='Direção'),
         current_magnitude=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='Magnitude I (pu)'),
         current_angle=widgets.IntSlider(min=-180, max=180, step=5, value=0, description='Ângulo I (°)'),
         save_image=widgets.Checkbox(value=False, description='Salvar Imagem (PNG)'))


interactive(children=(Dropdown(description='Fase de Falta', options=('A', 'B', 'C'), value='A'), IntSlider(val…

<function __main__.plot_ansi67(fault_phase, amt, pickup_current, direction, current_magnitude, current_angle, save_image)>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, Button, HBox, VBox
from matplotlib.patches import Wedge

def plot_ansi67_three_elements(fault_phase, amt, pickup_current1, pickup_current2, pickup_current3, direction1, direction2, direction3, current_magnitude, current_angle, element1_enabled, element2_enabled, element3_enabled, save_image):
    # Definição das tensões trifásicas de fase (sequência direta ABC)
    phase_voltages = {'A': np.exp(1j * 0), 'B': np.exp(1j * -2*np.pi/3), 'C': np.exp(1j * 2*np.pi/3)}
    
    # Definição das tensões de polarização
    polarization_mapping = {'A': ('Vbc', phase_voltages['B'] - phase_voltages['C']),
                            'B': ('Vca', phase_voltages['C'] - phase_voltages['A']),
                            'C': ('Vab', phase_voltages['A'] - phase_voltages['B'])}
    polarization_voltage, pol_voltage = polarization_mapping[fault_phase]
    
    # Rotacionando os fasores para que a tensão de polarização fique em 0 graus
    rotation_factor = np.conj(pol_voltage) / abs(pol_voltage)  # Rotação para alinhar a tensão de polarização em 0°
    rotated_voltages = {phase: voltage * rotation_factor for phase, voltage in phase_voltages.items()}
    
    # Criando círculo unitário para visualização da zona de atuação
    theta = np.linspace(0, 2 * np.pi, 300)
    x_circle = np.cos(theta)
    y_circle = np.sin(theta)
    
    # Ângulo de máxima sensibilidade (AMT) contado a partir da tensão de polarização
    amt_rad = np.radians(amt)
    
    # Definição da região de atuação (forward ou backward) para cada elemento
    def calculate_zone(direction, pickup_current):
        if direction == 'Forward':
            return np.linspace(amt_rad - np.pi/2, amt_rad + np.pi/2, 100)
        else:  # Backward
            return np.linspace(amt_rad + np.pi/2, amt_rad + 3*np.pi/2, 100)
    
    # Zonas de atuação para cada elemento
    theta_trip1 = calculate_zone(direction1, pickup_current1)
    theta_trip2 = calculate_zone(direction2, pickup_current2)
    theta_trip3 = calculate_zone(direction3, pickup_current3)
    
    # Cálculo dos vetores de corrente inserida
    current_angle_rad = np.radians(current_angle)
    current_vector = current_magnitude * np.exp(1j * current_angle_rad)
    rotated_current = current_vector * rotation_factor  # Aplicando rotação para manter a referência correta
    
    # Plot
    fig, ax = plt.subplots(figsize=(10,10))
    ax.plot(x_circle, y_circle, 'k--', alpha=0.5, label='Círculo Unitário')
    
    # Desenhando as zonas de proteção para os três elementos
    colors = ['#FF6347', '#32CD32', '#1E90FF']  # Cores distintas para cada elemento
    directions = [direction1, direction2, direction3]
    pickup_currents = [pickup_current1, pickup_current2, pickup_current3]
    zones = [theta_trip1, theta_trip2, theta_trip3]
    elements_enabled = [element1_enabled, element2_enabled, element3_enabled]
    labels = ['Elemento 1', 'Elemento 2', 'Elemento 3']
    
    for i in range(3):
        if elements_enabled[i]:
            x_trip = pickup_currents[i] * np.cos(zones[i])
            y_trip = pickup_currents[i] * np.sin(zones[i])
            ax.fill(x_trip, y_trip, colors[i], alpha=0.3, label=f'Zona de Trip {labels[i]}')
    
    # Plotando as tensões de fase rotacionadas
    for phase, voltage in rotated_voltages.items():
        ax.arrow(0, 0, voltage.real, voltage.imag, head_width=0.05, head_length=0.05, fc='gray', ec='gray', alpha=0.7)
        ax.text(1.1 * voltage.real, 1.1 * voltage.imag, f'V{phase}', fontsize=12, color='gray')
    
    # Destacando a tensão de polarização (agora fixa em 0°)
    ax.arrow(0, 0, 1, 0, head_width=0.05, head_length=0.05, fc='purple', ec='purple', label=f'Tensão de Polarização ({polarization_voltage})')
    ax.text(1.1, 0.1, polarization_voltage, fontsize=12, color='purple')
    
    # Inserindo o vetor de corrente
    ax.arrow(0, 0, rotated_current.real, rotated_current.imag, head_width=0.05, head_length=0.05, fc='green', ec='green', label='Corrente Inserida')
    ax.text(1.1 * rotated_current.real, 1.1 * rotated_current.imag, 'I', fontsize=12, color='green')
    
    # Configuração do gráfico
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_xlabel('Componente Real')
    ax.set_ylabel('Componente Imaginária')
    ax.set_title(f'Proteção ANSI 67 - Falta na Fase {fault_phase} ({polarization_voltage})')
    ax.legend()
    ax.grid()
    
    # Salvar a imagem apenas quando o botão for pressionado
    if save_image:
        plt.savefig('proteção_ANSI_67_three_elements.png', dpi=300, bbox_inches='tight')
        print("Imagem salva como 'proteção_ANSI_67_three_elements.png'")
    
    plt.show()

# Botão de salvar
save_button = Button(description="Salvar Imagem", button_style='success')

# Função para ativar a salvamento ao pressionar o botão
def on_save_button_click(b):
    plot_ansi67_three_elements(fault_phase='A', amt=0, pickup_current1=0.5, pickup_current2=0.5, pickup_current3=0.5, 
                               direction1='Forward', direction2='Forward', direction3='Forward', 
                               current_magnitude=1.0, current_angle=0, 
                               element1_enabled=True, element2_enabled=True, element3_enabled=True, 
                               save_image=True)

save_button.on_click(on_save_button_click)

# Widgets interativos
interact(plot_ansi67_three_elements, 
         fault_phase=widgets.Dropdown(options=['A', 'B', 'C'], value='A', description='Fase de Falta'),
         amt=widgets.IntSlider(min=-90, max=90, step=5, value=45, description='AMT (°)'),
         pickup_current1=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='Pick-up 1 (pu)'),
         pickup_current2=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=0.75, description='Pick-up 2 (pu)'),
         pickup_current3=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=0.5, description='Pick-up 3 (pu)'),
         direction1=widgets.RadioButtons(options=['Forward', 'Backward'], value='Forward', description='Direção 1'),
         direction2=widgets.RadioButtons(options=['Forward', 'Backward'], value='Backward', description='Direção 2'),
         direction3=widgets.RadioButtons(options=['Forward', 'Backward'], value='Forward', description='Direção 3'),
         current_magnitude=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='Magnitude I (pu)'),
         current_angle=widgets.IntSlider(min=-180, max=180, step=5, value=0, description='Ângulo I (°)'),
         element1_enabled=widgets.Checkbox(value=True, description='Ativar Elemento 1'),
         element2_enabled=widgets.Checkbox(value=True, description='Ativar Elemento 2'),
         element3_enabled=widgets.Checkbox(value=True, description='Ativar Elemento 3'),
         save_image=widgets.Checkbox(value=False, description='Salvar Imagem'))

# Exibindo o botão de salvar
display(save_button)




interactive(children=(Dropdown(description='Fase de Falta', options=('A', 'B', 'C'), value='A'), IntSlider(val…

Button(button_style='success', description='Salvar Imagem', style=ButtonStyle())